# Strava segment data

Choose some activity ID or the LAST one if `LAST=True`

In [1]:
activity=4310029237#4088494919
date='November 8, 2020'
LAST=True

In [2]:
import helium
import pandas as pd
import os
import time
import sys
def metadata(**kwargs):
    return kwargs
db_dir='db'
db_file='{}/strava.json'.format(db_dir)

In [3]:
bkp=pd.read_json(db_file)
if not LAST and not bkp[bkp.Info.astype(str).str.contains(date)].empty:
    sys.exit('Check if already exists in bkp')

In [4]:
os.makedirs(db_dir,exist_ok=True)

if os.path.isfile(db_file):
    df=pd.read_json(db_file)
else:
    df=pd.DataFrame()

browser=helium.start_chrome("https://www.strava.com/login")

helium.write('diego.restrepo@gmail.com',into='Your Email')

helium.write('opoqdh11',into='Password')

helium.click('Log In')

time.sleep(10)
browser.maximize_window()

if LAST:
    #Required to activate card-section below
    lt=[t for t in browser.find_elements_by_class_name('card-section') 
       if t.text.find('Latest Activity')>-1][0]

    lt.find_element_by_tag_name('a').click()

else:
    browser.get('https://www.strava.com/activities/{}'.format(activity))
# Web page with latest activity

# General info

time.sleep(10)
dt=browser.find_element_by_class_name('details')

date=dt.find_element_by_tag_name('time').text

name=dt.find_element_by_tag_name('h1').text

kk=browser.find_elements_by_tag_name("ul")

q=[ k for k in kk if k.text.find('\nDistance')>-1][0]

distance,time,elevation=(t.text for t in q.find_elements_by_tag_name('strong') )

q=[ k for k in kk if k.text.find('\nEstimated Avg Power')>-1][0]

power,energy=(t.text for t in q.find_elements_by_tag_name('strong') )

# Load tables

HIDDEN=True
try:
    browser.find_element_by_id('show-hidden-efforts').click()
except:
    HIDDEN=False
    pass

html=browser.page_source

ll=pd.read_html(html)

# General info table

mtdt=ll[0].groupby('Unnamed: 0').sum().transpose().to_dict()

mtdt.update(  metadata(date=date,name=name,distance=distance,time=time,
         elevation=elevation,power=elevation,energy=energy)
   )

#Fix time
import time

# Segments

sgmt=ll[1][['Name','Time','Speed','Power']].reset_index(drop=True)

if HIDDEN:
    sgmt=sgmt.append( ll[2].rename({2:'Name',4:'Time',5:'Speed',6:'Power'},
            axis='columns')[['Name','Time','Speed','Power']] 
           ).reset_index(drop=True)

#Add general info
sgmt['Info']=''

sgmt['Info'] = [ mtdt  for i in sgmt.index]

In [5]:
ll[0]

,Unnamed: 0,Avg,Max
0,Speed,14.1km/h,76.7km/h
1,Calories,6082,6082
2,Elapsed Time,8:53:04,8:53:04


In [6]:
sgmt

,Name,Time,Speed,Power,Info
0,Semáforo Kia - Bomba Esso 0.63km 10m 1.5%,2:05,18.2km/h,262W,"{'Calories': {'Avg': '6082', 'Max': '6082'}, '..."
1,semaforo Mayorca- Semaforo zona Norte 1 0.46k...,1:08,24.6km/h,222W,"{'Calories': {'Avg': '6082', 'Max': '6082'}, '..."
2,Ciclovia Vegas 1.66km 26m 1.3%,4:32,22.0km/h,259W,"{'Calories': {'Avg': '6082', 'Max': '6082'}, '..."
3,Bomba Terpel - Semáforo Cll 69sur 0.97km 16m...,2:36,22.5km/h,285W,"{'Calories': {'Avg': '6082', 'Max': '6082'}, '..."
4,Semaforo zona Norte 2 - Semaforo cruce 0.37km...,1:11,18.9km/h,225W,"{'Calories': {'Avg': '6082', 'Max': '6082'}, '..."
...,...,...,...,...,...
154,NaN,NaN,1:12:57,5.7km/h,"{'Calories': {'Avg': '6082', 'Max': '6082'}, '..."
155,NaN,NaN,1:13:17,5.7km/h,"{'Calories': {'Avg': '6082', 'Max': '6082'}, '..."
156,NaN,NaN,5:12,6.5km/h,"{'Calories': {'Avg': '6082', 'Max': '6082'}, '..."
157,NaN,NaN,13:46,6.4km/h,"{'Calories': {'Avg': '6082', 'Max': '6082'}, '..."


In [7]:
df=df.append(sgmt,sort=False).reset_index(drop=True)

In [8]:
df.to_json(db_file)

### Save notebook here

In [9]:
f=open('commit.sh','w')
f.write('git commit -am "{}"\n'.format(
    df.iloc[-1]['Info']['name']+' '+df.iloc[-1]['Info']['date']
))
f.write('git push origin master')
f.close()

In [10]:
!bash commit.sh

[master 46a6802] Morning Ride 5:31 AM on Sunday, November 29, 2020
 2 files changed, 378 insertions(+), 13 deletions(-)
Enumerating objects: 9, done.
Counting objects: 100% (9/9), done.
Delta compression using up to 8 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (5/5), 9.05 KiB | 926.00 KiB/s, done.
Total 5 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To github.com:restrepo/strava.git
   752a663..46a6802  master -> master


In [11]:
if not( input('Close browser?').find('n')>-1 ):
    browser.close()

Close browser? 
